In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, LSTM
from pathlib import Path
import os
import warnings
warnings.filterwarnings('ignore')

# Diretório onde o script está localizado
base_diretório = Path(os.getcwd())
diretório_princapl = base_diretório.parent
caminho = diretório_princapl / 'Bases' / 'BNB Historical Data.csv'

# Ler o arquivo ano.txt e pegar o ano
# with open(diretório_princapl/'data'/'ano.txt', 'r') as file:
#     ano = int(file.read().strip()) 
ano = 2020

# Ler arquivo da base
acao = pd.read_csv(caminho)

# Formatar data
acao['Date'] = pd.to_datetime(acao['Date'])

# Separar um df no qual vai conter apenas os dado para treinamento
# Que é com base no ano escolhido pelo jogador
df_treinamento = acao[acao['Date'].dt.year < ano]

# Apenas cotação dos dados de treinamento 
cotacao_treinamento = df_treinamento['Price'].to_numpy().reshape(-1, 1)

# Cotação dos dados da base toda
cotacao = acao['Price'].to_numpy().reshape(-1, 1)

# Armazenar tamanho dos dados de treinamento
tamanho_dados_treinamento = int(len(cotacao_treinamento) * 1)

# Escalar os dados entre 0 e 1
escalador = MinMaxScaler(feature_range=(0, 1))
dados_entre_0_e_1_treinamento = escalador.fit_transform(cotacao[0:tamanho_dados_treinamento, :])
dados_entre_0_e_1_teste = escalador.transform(cotacao[tamanho_dados_treinamento:, :])

dados_entre_0_e_1 = np.concatenate((dados_entre_0_e_1_treinamento, dados_entre_0_e_1_teste), axis=0)

dados_para_treinamento = dados_entre_0_e_1[0:tamanho_dados_treinamento, :]
treinamento_x, treinamento_y = [], []

for i in range(60, len(dados_para_treinamento)):
    treinamento_x.append(dados_para_treinamento[i - 60:i, 0])
    treinamento_y.append(dados_para_treinamento[i, 0])

# Transformando listas em arrays
treinamento_x, treinamento_y = np.array(treinamento_x), np.array(treinamento_y)
treinamento_x = treinamento_x.reshape(treinamento_x.shape[0], treinamento_x.shape[1], 1)


# produto cartesiano
# Parâmetros para otimização
neurons_options = [50, 60, 70, 80, 90, 100]
batch_size_options = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120]
epochs_options = [10, 20]

results = []

rodada = 1
# rodar 30x as 144 combinações possíveis

for neurons in neurons_options:
    for batch_size in batch_size_options:
        for epochs in epochs_options:
            while rodada <= 30:        
                print(f"Combinação {rodada}: neuronios={neurons}, batch_size={batch_size}, epochs_options={epochs}")
                    
                # Construindo o modelo
                modelo = Sequential()
                modelo.add(LSTM(neurons, return_sequences=True, input_shape=(treinamento_x.shape[1], 1)))
                modelo.add(LSTM(neurons // 2, return_sequences=False))
                modelo.add(Dense((neurons // 2)//2))
                modelo.add(Dense(1))
                # Compilando o modelo
                modelo.compile(optimizer="adam", loss="mean_squared_error")
                    
                # Treinando o modelo
                modelo.fit(treinamento_x, treinamento_y, batch_size=batch_size, epochs=epochs, verbose=0)
                # Criando dados de teste
                dados_teste = dados_entre_0_e_1[tamanho_dados_treinamento - 60:, :]
                teste_x = []
                teste_y = cotacao[tamanho_dados_treinamento:, :]
                for i in range(60, len(dados_teste)):
                    teste_x.append(dados_teste[i - 60:i, 0])
                teste_x = np.array(teste_x).reshape(len(teste_x), 60, 1)
                # Pegando predições do modelo
                predicoes = modelo.predict(teste_x)
                predicoes = escalador.inverse_transform(predicoes)
                    
                # Criar df
                df_previsao = pd.DataFrame({"Date": acao['Date'].iloc[tamanho_dados_treinamento:],
                                                "Price": acao['Price'].iloc[tamanho_dados_treinamento:],
                                                "predicoes": predicoes.reshape(len(predicoes))
                                            })
                    
                df_previsao.set_index('Date', inplace=True)
                    
                df_previsao['Semana'] = ((df_previsao.index - df_previsao.index.min()).days // 7) + 1
                    
                #========================== SEMANAL ==========================
                df_semana = df_previsao
                # Agrupar por semana e calcular o preço médio semanal para real e previsões
                df_semana = df_previsao.groupby('Semana').agg({
                    'Price': 'mean',        # Preço real médio por semana
                    'predicoes': 'mean'     # Previsão média por semana
                })
                # Calcular a variação percentual semanal para Price e Predicoes
                df_semana['Variação Real (%)'] = df_semana['Price'].pct_change() * 100
                df_semana['Variação Prevista (%)'] = df_semana['predicoes'].pct_change() * 100
                # Remover NaN (primeira linha não terá variação por não ter valor anterior)
                df_semana = df_semana.dropna()
                # Calcular se a direção do valor previsto foi igual ao real (1 = acertou | 0 = errou)
                df_semana['direcao_correta'] = np.where(
                    (df_semana['Variação Real (%)'] > 0) & (df_semana['Variação Prevista (%)'] > 0) |
                    (df_semana['Variação Real (%)'] < 0) & (df_semana['Variação Prevista (%)'] < 0),
                    1, 0
                )
                # Calcular a métrica de Erro de Direção (Directional Accuracy)
                directional_accuracy_semanal = df_semana['direcao_correta'].mean()
                directional_accuracy_semanal = round(directional_accuracy_semanal * 100, 2)
                # Avaliando o modelo
                mae = mean_absolute_error(teste_y, predicoes)
                mape = mean_absolute_percentage_error(teste_y, predicoes)
                rmse = mean_squared_error(teste_y, predicoes, squared=False)
                # Armazenando os resultados
                results.append({
                    'neurons': neurons,
                    'batch_size': batch_size,
                    'epochs': epochs,
                    'MAE': mae,
                    'MAPE': mape,
                    'RMSE': rmse,
                    'directional_accuracy_semanal': directional_accuracy_semanal
                })

    rodada += 1

# Convertendo resultados em DataFrame
results_df = pd.DataFrame(results)

# Exibindo resultados
print(results_df)

Combinação 1: neuronios=50, batch_size=10, epochs_options=10
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
Combinação 1: neuronios=50, batch_size=10, epochs_options=20
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Combinação 1: neuronios=50, batch_size=20, epochs_options=10
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
Combinação 1: neuronios=50, batch_size=20, epochs_options=20
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
Combinação 1: neuronios=50, batch_size=30, epochs_options=10
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Combinação 1: neuronios=50, batch_size=30, epochs_options=20
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Combinação 1: neuronios=50, batch_size=40, epochs_options=10
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
Combinação 1: neuronios=50, batch_size=40, epochs_options=20
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Combinação 1: neuronios=50, batch_size=50, epochs_options=10
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Combinação 1: neuronios=50, batch_size=50, epochs_options=20
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


: 

In [2]:
results_df.to_excel("Resultados testes parametros/BNB - Resultado geral.xlsx", engine="openpyxl", index=False)

# Agrupar pelas combinações de parâmetros e calcular a média das métricas
resultados_agrupados = results_df.groupby(['neurons', 'batch_size', 'epochs']).agg({
    'MAE': 'mean',
    'MAPE': 'mean',
    'RMSE': 'mean',
    'directional_accuracy_semanal': 'mean'
}).reset_index()

# Exibir os resultados agrupados
print(resultados_agrupados)

# Gráfico do melhor modelo (opcional)
best_model = resultados_agrupados.loc[resultados_agrupados['directional_accuracy_semanal'].idxmax()]

print(f"\nMelhor Modelo: Neurônios={best_model['neurons']}, batch_size={best_model['batch_size']}, Epochs={best_model['epochs']}")

resultados_agrupados.to_excel("Resultados testes parametros/BNB - resultado geral agrupado.xlsx", engine="openpyxl", index=False)

     neurons  batch_size  epochs         MAE      MAPE        RMSE  \
0         50          10      10  158.742580  0.414932  208.738960   
1         50          10      20  146.468996  0.382222  191.784031   
2         50          20      10  179.307994  0.475191  231.390414   
3         50          20      20  131.969745  0.336836  180.208544   
4         50          30      10  190.749069  0.510877  244.119677   
..       ...         ...     ...         ...       ...         ...   
139      100         100      20  166.633845  0.437480  218.936396   
140      100         110      10  185.244941  0.494097  238.362708   
141      100         110      20  159.738307  0.419624  210.677456   
142      100         120      10  196.488405  0.528834  249.733035   
143      100         120      20  166.979799  0.437518  219.822088   

     directional_accuracy_semanal  
0                           80.25  
1                           81.48  
2                           62.55  
3              